In [0]:
!pip install kaggle

     |████████████████████████████████| 58 kB 1.1 MB/s 
     |████████████████████████████████| 78 kB 3.0 MB/s 
     |████████████████████████████████| 78 kB 7.5 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73053 sha256=994737652d6eaa94b0562acbe8f14de92f094aeb1f54e5dc9c03c8906081068b
  Stored in directory: /root/.cache/pip/wheels/29/da/11/144cc25aebdaeb4931b231e25fd34b394e6a5725cbb2f50106
Successfully built kaggle
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


Create an api token and upload it here.

In [0]:
%sh mkdir -p /root/.kaggle

Create a Kaggle API token for downloading the data.
See https://www.kaggle.com/docs/api

In [0]:
import json

# paste your token here
token = {"username":"","key":""}

with open('/root/.kaggle/kaggle.json', 'w') as f:
    json.dump(token, f)

In [0]:
!kaggle datasets download -d "yuanyuwendymu/airline-delay-and-cancellation-data-2009-2018"

100%|█████████████████████████████████████▉| 1.94G/1.95G [00:27<00:00, 42.9MB/s]
100%|██████████████████████████████████████| 1.95G/1.95G [00:27<00:00, 74.7MB/s]


In [0]:
!unzip airline-delay-and-cancellation-data-2009-2018.zip

Archive:  airline-delay-and-cancellation-data-2009-2018.zip
  inflating: 2009.csv                
  inflating: 2010.csv                
  inflating: 2011.csv                
  inflating: 2012.csv                
  inflating: 2013.csv                
  inflating: 2014.csv                
  inflating: 2015.csv                
  inflating: 2016.csv                
  inflating: 2017.csv                
  inflating: 2018.csv                


In [0]:
!du -ch {2009..2015}.csv

756M	2009.csv
758M	2010.csv
713M	2011.csv
740M	2012.csv
751M	2013.csv
687M	2014.csv
686M	2015.csv
5.0G	total


In [0]:
dbutils.fs.mkdirs("dbfs:/user/airline")

Out[21]: True

In [0]:
df = spark.read.format("csv").option("header","true").load("file:/databricks/driver/{2009,2010,2011,2012,2013,2014,2015}.csv")

In [0]:
df.printSchema()

root
 |-- FL_DATE: string (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- OP_CARRIER_FL_NUM: string (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- CRS_DEP_TIME: string (nullable = true)
 |-- DEP_TIME: string (nullable = true)
 |-- DEP_DELAY: string (nullable = true)
 |-- TAXI_OUT: string (nullable = true)
 |-- WHEELS_OFF: string (nullable = true)
 |-- WHEELS_ON: string (nullable = true)
 |-- TAXI_IN: string (nullable = true)
 |-- CRS_ARR_TIME: string (nullable = true)
 |-- ARR_TIME: string (nullable = true)
 |-- ARR_DELAY: string (nullable = true)
 |-- CANCELLED: string (nullable = true)
 |-- CANCELLATION_CODE: string (nullable = true)
 |-- DIVERTED: string (nullable = true)
 |-- CRS_ELAPSED_TIME: string (nullable = true)
 |-- ACTUAL_ELAPSED_TIME: string (nullable = true)
 |-- AIR_TIME: string (nullable = true)
 |-- DISTANCE: string (nullable = true)
 |-- CARRIER_DELAY: string (nullable = true)
 |-- WEATHER_DELAY: strin

In [0]:
# dropping the unnamed column right away
df = df.drop("Unnamed: 27")

In [0]:
display(df.limit(10))

FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
2009-01-01,XE,1204,DCA,EWR,1100,1058.0,-2.0,18.0,1116.0,1158.0,8.0,1202,1206.0,4.0,0.0,null,0.0,62.0,68.0,42.0,199.0,null,null,null,null,null
2009-01-01,XE,1206,EWR,IAD,1510,1509.0,-1.0,28.0,1537.0,1620.0,4.0,1632,1624.0,-8.0,0.0,null,0.0,82.0,75.0,43.0,213.0,null,null,null,null,null
2009-01-01,XE,1207,EWR,DCA,1100,1059.0,-1.0,20.0,1119.0,1155.0,6.0,1210,1201.0,-9.0,0.0,null,0.0,70.0,62.0,36.0,199.0,null,null,null,null,null
2009-01-01,XE,1208,DCA,EWR,1240,1249.0,9.0,10.0,1259.0,1336.0,9.0,1357,1345.0,-12.0,0.0,null,0.0,77.0,56.0,37.0,199.0,null,null,null,null,null
2009-01-01,XE,1209,IAD,EWR,1715,1705.0,-10.0,24.0,1729.0,1809.0,13.0,1900,1822.0,-38.0,0.0,null,0.0,105.0,77.0,40.0,213.0,null,null,null,null,null
2009-01-01,XE,1212,ATL,EWR,1915,1913.0,-2.0,19.0,1932.0,2108.0,15.0,2142,2123.0,-19.0,0.0,null,0.0,147.0,130.0,96.0,745.0,null,null,null,null,null
2009-01-01,XE,1212,CLE,ATL,1645,1637.0,-8.0,12.0,1649.0,1820.0,5.0,1842,1825.0,-17.0,0.0,null,0.0,117.0,108.0,91.0,554.0,null,null,null,null,null
2009-01-01,XE,1214,DCA,EWR,1915,1908.0,-7.0,9.0,1917.0,1953.0,34.0,2035,2027.0,-8.0,0.0,null,0.0,80.0,79.0,36.0,199.0,null,null,null,null,null
2009-01-01,XE,1215,EWR,DCA,1715,1710.0,-5.0,28.0,1738.0,1819.0,4.0,1838,1823.0,-15.0,0.0,null,0.0,83.0,73.0,41.0,199.0,null,null,null,null,null
2009-01-01,XE,1217,EWR,DCA,1300,1255.0,-5.0,15.0,1310.0,1349.0,7.0,1408,1356.0,-12.0,0.0,null,0.0,68.0,61.0,39.0,199.0,null,null,null,null,null


In [0]:
df.write.format("delta").mode("overwrite").save("dbfs:/user/airline/table")

In [0]:
# how to load the data:
test_df = spark.read.format("delta").load("dbfs:/user/airline/table")
display(test_df.limit(10))

FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
2009-01-01,XE,1204,DCA,EWR,1100,1058.0,-2.0,18.0,1116.0,1158.0,8.0,1202,1206.0,4.0,0.0,null,0.0,62.0,68.0,42.0,199.0,null,null,null,null,null
2009-01-01,XE,1206,EWR,IAD,1510,1509.0,-1.0,28.0,1537.0,1620.0,4.0,1632,1624.0,-8.0,0.0,null,0.0,82.0,75.0,43.0,213.0,null,null,null,null,null
2009-01-01,XE,1207,EWR,DCA,1100,1059.0,-1.0,20.0,1119.0,1155.0,6.0,1210,1201.0,-9.0,0.0,null,0.0,70.0,62.0,36.0,199.0,null,null,null,null,null
2009-01-01,XE,1208,DCA,EWR,1240,1249.0,9.0,10.0,1259.0,1336.0,9.0,1357,1345.0,-12.0,0.0,null,0.0,77.0,56.0,37.0,199.0,null,null,null,null,null
2009-01-01,XE,1209,IAD,EWR,1715,1705.0,-10.0,24.0,1729.0,1809.0,13.0,1900,1822.0,-38.0,0.0,null,0.0,105.0,77.0,40.0,213.0,null,null,null,null,null
2009-01-01,XE,1212,ATL,EWR,1915,1913.0,-2.0,19.0,1932.0,2108.0,15.0,2142,2123.0,-19.0,0.0,null,0.0,147.0,130.0,96.0,745.0,null,null,null,null,null
2009-01-01,XE,1212,CLE,ATL,1645,1637.0,-8.0,12.0,1649.0,1820.0,5.0,1842,1825.0,-17.0,0.0,null,0.0,117.0,108.0,91.0,554.0,null,null,null,null,null
2009-01-01,XE,1214,DCA,EWR,1915,1908.0,-7.0,9.0,1917.0,1953.0,34.0,2035,2027.0,-8.0,0.0,null,0.0,80.0,79.0,36.0,199.0,null,null,null,null,null
2009-01-01,XE,1215,EWR,DCA,1715,1710.0,-5.0,28.0,1738.0,1819.0,4.0,1838,1823.0,-15.0,0.0,null,0.0,83.0,73.0,41.0,199.0,null,null,null,null,null
2009-01-01,XE,1217,EWR,DCA,1300,1255.0,-5.0,15.0,1310.0,1349.0,7.0,1408,1356.0,-12.0,0.0,null,0.0,68.0,61.0,39.0,199.0,null,null,null,null,null
